In [ ]:
!pip install transformers
# install huggingface transformers library

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.3 MB/s 
     |████████████████████████████████| 6.6 MB 28.5 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 596 kB 51.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
WAV_FILENAME = "/content/drive/My Drive/test2.wav"

In [ ]:
import os
filenames = os.listdir("/content/drive/My Drive/research/audio")
filenames.sort() # list of filenames

['000_190105.wav', '001_190105.wav', '002_190105.wav', '003_190105.wav', '004_190105.wav', '005_190105.wav', '006_190105.wav', '007_190105.wav', '008_190105.wav', '009_190105.wav', '010_190105.wav', '011_190105.wav']


In [ ]:
from transformers import pipeline
# This will work on any of the thousands of models at
# https://huggingface.co/models?pipeline_tag=automatic-speech-recognition
pipe = pipeline(model="facebook/wav2vec2-base-960h")

transcriptions = {}

for filename in filenames:
  whole_path = f"/content/drive/My Drive/research/audio/{filename}"
  output = pipe(whole_path, chunk_length_s=10, stride_length_s=(4, 2))
  # print(output)
  transcriptions[filename[:-4]] = output['text'].lower()
  print(f"transcription for {filename}:")
  print(transcriptions[filename[:-4]])
  print("")

Downloading:   0%|          | 0.00/2.79k [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


transcription for 000_190105.wav:
fifty five yearl mail found unconscious i'm i a driverside a passenger seat of his car his wife reported that he snorted a line of heroine before just prior to losing consciousness patienters originally presented onresponsive and pale with shallow ineffective respirations at a rate of about five his heart rache was one eighteen his blood pressure was two hundred five over one nineteen his blood gluwcust levels one twenty sixt his o two saturations was nine were ninety four per cent pation required back mask vinhilation with not with attached oxygen however after zera point two five miligrams of nelocsone intervenously wpatient as now awake an breathing normally with a improvement vital signs and respiratory status it no longer need supplemental oxygen

transcription for 001_190105.wav:
ooh eames and fire were despatched to the behavior clinic for a fifty one year old female complaining of shortness a breath upon arriving on location we found the female

In [ ]:
!pip install evaluate
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.6 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149864 sha256=331935be56b697a1fba4ad4e4fad0c08ead7c719b095c6f50ced6bbf396e3f1c
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [30]:
# calculate WER for each sample file
from evaluate import load

avg_score = 0
for filename in filenames:
  model_output = transcriptions[filename[:-4]]

  correct_output_filename = f"/content/drive/My Drive/research/transcripts/{filename[:-4]}.txt"
  # print(correct_output_filename)
  with open(correct_output_filename, 'r') as f:
    correct_output = f.readlines()
  wer = load("wer")
  # print(correct_output)
  # print(model_output)
  wer_score = wer.compute(predictions=[model_output], references=correct_output)
  print(filename)
  print(f"correct output: {correct_output}")
  print(f"model output: {model_output}")
  print(f"WER: {wer_score}\n\n")
  avg_score += wer_score


avg_score /= len(filenames)

000_190105.wav
correct output: ['55 year old male found unconscious  driver side passenger seat of his car his wife reported that he snorted a line of heroin before just prior to losing to consciousness patient originally presented unresponsive and pale with shallow ineffective respirations at a rate of about 5 with heart rate was 118 his blood pressure was 205 over 119 his blood glucose levels 126 his O2 saturations was were 94% patient required bag mask ventilation with with attached oxygen however after 0.25 mg of naloxone intravenously patient is now awake and breathing normally with Improvement in Vital Signs and respiratory status and no longer needs supplemental oxygen']
model output: fifty five yearl mail found unconscious i'm i a driverside a passenger seat of his car his wife reported that he snorted a line of heroine before just prior to losing consciousness patienters originally presented onresponsive and pale with shallow ineffective respirations at a rate of about five hi

KeyboardInterrupt: ignored

In [ ]:
print(f"Average WER: {avg_score}")

Average WER: 0.40577027804138655
